##### The dataset was supplied through kaggle *https://www.kaggle.com/datasets/jtrotman/formula-1-race-data?select=results.csv*

In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [13]:
data = pd.read_csv("results.csv")

data[0:1]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [10]:
missing_values_count = data.isnull().sum()

missing_values_count

resultId           0
raceId             0
driverId           0
constructorId      0
number             0
grid               0
position           0
positionText       0
positionOrder      0
points             0
laps               0
time               0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
statusId           0
dtype: int64